# Missing Data Analysis - Habitable Planet Hunter Dataset

**Issue #57**: Identify the extent of missing data in each of the 30 approved columns.

This notebook analyzes:
- What data is missing in each column
- How much data is missing (count and percentage)
- Relationships between missing values across different columns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('../datasets/full_data.csv')
print(f"Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")

In [ ]:
# Define the 30 approved columns as per README.md
approved_columns = [
    # Planet Physical Properties (1-6)
    'P_MASS', 'P_RADIUS', 'P_DENSITY', 'P_GRAVITY', 'P_ESCAPE', 'P_TYPE',
    # Planet Orbital Parameters (7-15)
    'P_PERIOD', 'P_SEMI_MAJOR_AXIS', 'P_ECCENTRICITY', 'P_INCLINATION', 
    'P_OMEGA', 'P_PERIASTRON', 'P_APASTRON', 'P_IMPACT_PARAMETER', 'P_HILL_SPHERE',
    # Stellar Properties (16-26)
    'S_MASS', 'S_RADIUS', 'S_LUMINOSITY', 'S_TEMPERATURE', 'S_AGE', 
    'S_METALLICITY', 'S_LOG_G', 'S_TYPE', 'S_MAG', 'S_DISC', 'S_MAGNETIC_FIELD',
    # System & Meta Data (27-30)
    'S_SNOW_LINE', 'S_TIDAL_LOCK', 'P_DETECTION', 'P_DISTANCE'
]

# Verify all columns exist in dataset
missing_cols = [col for col in approved_columns if col not in df.columns]
if missing_cols:
    print(f"Warning: Columns not found in dataset: {missing_cols}")
else:
    print(f"All {len(approved_columns)} approved columns found in dataset.")

In [ ]:
# Extract only the 30 approved columns for analysis
df_approved = df[approved_columns].copy()
print(f"Analyzing {df_approved.shape[1]} columns with {df_approved.shape[0]} rows")

---
## 1. Missing Data Summary

Overview of missing values for each of the 30 approved columns.

In [ ]:
# Calculate missing data statistics
total_rows = len(df_approved)

missing_stats = pd.DataFrame({
    'Column': approved_columns,
    'Missing Count': [df_approved[col].isna().sum() for col in approved_columns],
    'Missing %': [round(df_approved[col].isna().sum() / total_rows * 100, 2) for col in approved_columns],
    'Present Count': [df_approved[col].notna().sum() for col in approved_columns],
    'Present %': [round(df_approved[col].notna().sum() / total_rows * 100, 2) for col in approved_columns],
    'Data Type': [df_approved[col].dtype for col in approved_columns]
})

# Sort by missing percentage (descending)
missing_stats_sorted = missing_stats.sort_values('Missing %', ascending=False).reset_index(drop=True)
missing_stats_sorted.index = missing_stats_sorted.index + 1
missing_stats_sorted

---
## 2. Missing Data by Category

Grouping the 30 columns by their category to understand patterns.

In [ ]:
# Define column categories
categories = {
    'Planet Physical Properties': ['P_MASS', 'P_RADIUS', 'P_DENSITY', 'P_GRAVITY', 'P_ESCAPE', 'P_TYPE'],
    'Planet Orbital Parameters': ['P_PERIOD', 'P_SEMI_MAJOR_AXIS', 'P_ECCENTRICITY', 'P_INCLINATION', 
                                   'P_OMEGA', 'P_PERIASTRON', 'P_APASTRON', 'P_IMPACT_PARAMETER', 'P_HILL_SPHERE'],
    'Stellar Properties': ['S_MASS', 'S_RADIUS', 'S_LUMINOSITY', 'S_TEMPERATURE', 'S_AGE', 
                           'S_METALLICITY', 'S_LOG_G', 'S_TYPE', 'S_MAG', 'S_DISC', 'S_MAGNETIC_FIELD'],
    'System & Meta Data': ['S_SNOW_LINE', 'S_TIDAL_LOCK', 'P_DETECTION', 'P_DISTANCE']
}

# Calculate category-level statistics
category_stats = []
for category, cols in categories.items():
    total_cells = len(cols) * total_rows
    missing_cells = df_approved[cols].isna().sum().sum()
    avg_missing = round(missing_cells / total_cells * 100, 2)
    category_stats.append({
        'Category': category,
        'Columns': len(cols),
        'Total Cells': total_cells,
        'Missing Cells': missing_cells,
        'Avg Missing %': avg_missing
    })

category_df = pd.DataFrame(category_stats)
category_df

---
## 3. Visualization: Missing Data Heatmap

In [ ]:
# Create missing data visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart of missing percentages
colors = ['#e74c3c' if x > 50 else '#f39c12' if x > 20 else '#27ae60' for x in missing_stats_sorted['Missing %']]
axes[0].barh(missing_stats_sorted['Column'], missing_stats_sorted['Missing %'], color=colors)
axes[0].set_xlabel('Missing Percentage (%)')
axes[0].set_ylabel('Column')
axes[0].set_title('Missing Data Percentage by Column')
axes[0].invert_yaxis()
axes[0].axvline(x=50, color='red', linestyle='--', alpha=0.5, label='50% threshold')
axes[0].axvline(x=20, color='orange', linestyle='--', alpha=0.5, label='20% threshold')
axes[0].legend()

# Category-level pie chart
axes[1].pie(category_df['Missing Cells'], labels=category_df['Category'], autopct='%1.1f%%', 
            colors=['#3498db', '#9b59b6', '#e67e22', '#1abc9c'])
axes[1].set_title('Distribution of Missing Values by Category')

plt.tight_layout()
plt.show()

---
## 4. Missing Data Correlation Analysis

Analyzing relationships between missing values across columns.

In [ ]:
# Create a binary matrix: 1 = missing, 0 = present
missing_matrix = df_approved.isna().astype(int)

# Calculate correlation between missing patterns
missing_corr = missing_matrix.corr()

# Plot correlation heatmap
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(missing_corr, dtype=bool))
sns.heatmap(missing_corr, mask=mask, annot=False, cmap='RdYlBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={'shrink': 0.8})
plt.title('Correlation Between Missing Values Across Columns\n(Higher values = columns tend to be missing together)')
plt.tight_layout()
plt.show()

In [ ]:
# Find highly correlated missing patterns (correlation > 0.5)
high_corr_pairs = []
for i in range(len(approved_columns)):
    for j in range(i+1, len(approved_columns)):
        corr_val = missing_corr.iloc[i, j]
        if abs(corr_val) > 0.5:
            high_corr_pairs.append({
                'Column 1': approved_columns[i],
                'Column 2': approved_columns[j],
                'Correlation': round(corr_val, 3)
            })

if high_corr_pairs:
    corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', ascending=False)
    print("Highly Correlated Missing Patterns (|correlation| > 0.5):")
    display(corr_df)
else:
    print("No highly correlated missing patterns found (threshold: 0.5)")

---
## 5. Missing Data Patterns Analysis

In [ ]:
# Analyze rows with most missing values
missing_per_row = df_approved.isna().sum(axis=1)

print("Missing Values Distribution Across Rows:")
print(f"  - Rows with 0 missing values: {(missing_per_row == 0).sum()}")
print(f"  - Rows with 1-5 missing values: {((missing_per_row >= 1) & (missing_per_row <= 5)).sum()}")
print(f"  - Rows with 6-10 missing values: {((missing_per_row >= 6) & (missing_per_row <= 10)).sum()}")
print(f"  - Rows with 11-15 missing values: {((missing_per_row >= 11) & (missing_per_row <= 15)).sum()}")
print(f"  - Rows with 16-20 missing values: {((missing_per_row >= 16) & (missing_per_row <= 20)).sum()}")
print(f"  - Rows with >20 missing values: {(missing_per_row > 20).sum()}")
print(f"\n  - Average missing values per row: {round(missing_per_row.mean(), 2)}")
print(f"  - Max missing values in a row: {missing_per_row.max()}")

In [ ]:
# Histogram of missing values per row
plt.figure(figsize=(10, 5))
plt.hist(missing_per_row, bins=range(0, 32), edgecolor='black', alpha=0.7, color='#3498db')
plt.xlabel('Number of Missing Values')
plt.ylabel('Number of Rows')
plt.title('Distribution of Missing Values Per Row (30 Approved Columns)')
plt.axvline(x=missing_per_row.mean(), color='red', linestyle='--', label=f'Mean: {missing_per_row.mean():.1f}')
plt.legend()
plt.tight_layout()
plt.show()

---
## 6. Detection Method Impact on Missing Data

In [ ]:
# Analyze missing data by detection method
if 'P_DETECTION' in df_approved.columns:
    detection_missing = df_approved.groupby('P_DETECTION').apply(
        lambda x: x.isna().sum().sum() / (len(x) * len(approved_columns)) * 100
    ).round(2)
    
    detection_counts = df_approved['P_DETECTION'].value_counts()
    
    detection_analysis = pd.DataFrame({
        'Detection Method': detection_missing.index,
        'Planet Count': [detection_counts.get(m, 0) for m in detection_missing.index],
        'Avg Missing %': detection_missing.values
    }).sort_values('Planet Count', ascending=False)
    
    print("Missing Data by Detection Method:")
    display(detection_analysis)

---
## 7. Summary Statistics

In [ ]:
# Final summary
total_cells = df_approved.shape[0] * df_approved.shape[1]
total_missing = df_approved.isna().sum().sum()
total_present = total_cells - total_missing

print("="*60)
print("MISSING DATA ANALYSIS SUMMARY")
print("="*60)
print(f"\nDataset Overview:")
print(f"  - Total Rows: {df_approved.shape[0]:,}")
print(f"  - Columns Analyzed: {df_approved.shape[1]}")
print(f"  - Total Data Points: {total_cells:,}")

print(f"\nMissing Data Overview:")
print(f"  - Total Missing Values: {total_missing:,}")
print(f"  - Total Present Values: {total_present:,}")
print(f"  - Overall Missing Rate: {round(total_missing/total_cells*100, 2)}%")

print(f"\nColumn-Level Statistics:")
print(f"  - Columns with 0% missing: {(missing_stats['Missing %'] == 0).sum()}")
print(f"  - Columns with <10% missing: {(missing_stats['Missing %'] < 10).sum()}")
print(f"  - Columns with 10-50% missing: {((missing_stats['Missing %'] >= 10) & (missing_stats['Missing %'] < 50)).sum()}")
print(f"  - Columns with ≥50% missing: {(missing_stats['Missing %'] >= 50).sum()}")

print(f"\nMost Complete Columns (Top 5):")
for idx, row in missing_stats.nsmallest(5, 'Missing %').iterrows():
    print(f"  - {row['Column']}: {row['Missing %']}% missing")

print(f"\nMost Incomplete Columns (Top 5):")
for idx, row in missing_stats.nlargest(5, 'Missing %').iterrows():
    print(f"  - {row['Column']}: {row['Missing %']}% missing")

print("\n" + "="*60)

---
## 8. Detailed Column Analysis Table

In [ ]:
# Create a comprehensive table with all details
detailed_analysis = missing_stats.copy()

# Add category information
def get_category(col):
    for cat, cols in categories.items():
        if col in cols:
            return cat
    return 'Unknown'

detailed_analysis['Category'] = detailed_analysis['Column'].apply(get_category)

# Reorder columns
detailed_analysis = detailed_analysis[['Column', 'Category', 'Data Type', 'Present Count', 'Present %', 'Missing Count', 'Missing %']]

# Style the dataframe
def highlight_missing(val):
    if isinstance(val, (int, float)):
        if val >= 50:
            return 'background-color: #ffcccc'
        elif val >= 20:
            return 'background-color: #fff3cd'
    return ''

styled_df = detailed_analysis.style.applymap(highlight_missing, subset=['Missing %'])
styled_df